## Терминология

- **Статистически значимый результат** — результат, который статистически значимо лучше 0.
- **Прокрас теста** — результат эксперимента статистически значимо отличается от 0, и у вас есть какой-то эффект.
- **Зелёный тест** — метрика в A/B-тесте статистически значимо стала лучше.
- **Красный тест** — метрика в A/B-тесте статистически значимо стала хуже.
- **Серый тест** — результат A/B-теста не статистически значим.
- **Тритмент** — фича или предложение, чьё воздействие на пользователей вы проверяете в A/B-тесте.
- **MDE** — минимальный детектируемый эффект. Размер, который должен иметь истинный эффект от тритмента, чтобы эксперимент его обнаружил с заданной долей уверенности (мощностью). Чем меньше MDE, тем лучше.
- **Мощность критерия** — вероятность критерия задетектировать эффект, если он действительно есть. Чем больше мощность критерия, тем он круче. 
- **Предпериод** — период до начала эксперимента.
- T - тестовая группа
- C - контрольная группа

Что важнее и предпочтительней?

1. При анализе A/B-тестов считайте не только p-value, но и доверительные интервалы с численными оценками эффекта. 
2. Считайте не только абсолютные метрики, но и относительные.

Алгоритм проверки статистических критериев

Идея простая:

1. Создаём как можно больше датасетов, поделённых на контроль и тест, без какого-либо различия между ними (обычный А/А-тест). 

2. Прогоняем на них придуманный критерий.

3. Если мы хотим, чтобы ошибка первого рода была 5%, то критерий должен ошибиться на этих примерах лишь в 5% случаев. То есть 0 не попал в доверительный интервал. 

## Абсолютный Т-тест

Построим абсолютный критерий следующим образом: будем проверять гипотезу $H_0 = \{\mathbb{E}(T) = \mathbb{E}(C)\}$, против сложной классической альтернативы $H_1 = \{\mathbb{E}(T) \neq \mathbb{E}(C)\}$. Для этого мы заведем функцию `absolute_ttest`, в которой на заданном уровне значимости $\alpha$ будем тестировать гипотезу о наличии эффекта.

In [16]:
# 1. Импорт всех нужных библиотек.
from collections import namedtuple
import scipy.stats as sps
import statsmodels.stats.api as sms
from tqdm.notebook import tqdm as tqdm_notebook # tqdm – библиотека для визуализации прогресса в цикле
from collections import defaultdict
from statsmodels.stats.proportion import proportion_confint
import numpy as np
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.5, palette='Set2')
ExperimentComparisonResults = namedtuple('ExperimentComparisonResults', 
                                        ['pvalue', 'effect', 'ci_length', 'left_bound', 'right_bound'])

# 2. Создание тестируемого критерия
def absolute_ttest(control, test, alpha=0.05):
    mean_control = np.mean(control)
    mean_test = np.mean(test)
    var_mean_control  = np.var(control) / len(control)
    var_mean_test  = np.var(test) / len(test)
    
    difference_mean = mean_test - mean_control
    difference_mean_var = var_mean_control + var_mean_test
    difference_distribution = sps.norm(loc=difference_mean, scale=np.sqrt(difference_mean_var))

    left_bound, right_bound = difference_distribution.ppf([alpha / 2, 1 - alpha / 2])
    ci_length = (right_bound - left_bound)
    pvalue = 2 * min(difference_distribution.cdf(0), difference_distribution.sf(0))
    effect = difference_mean
    return ExperimentComparisonResults(pvalue, effect, ci_length, left_bound, right_bound)

## Эксперимент 1

Здесь просто реализуем A/A тест с помощью функции, построенной выше.

In [17]:
def experiment_1(N=100000, size_control=1500, size_test=1500, loc_1=1, loc_2=1):
    # 3. Заводим счётчик.
    bad_cnt = 0

    # 4. Цикл проверки.
    for i in range(N):
        # 4.a. Тестирую A/A-тест.
        control = sps.expon.rvs(loc=loc_1, size=size_control)
        test = sps.expon.rvs(loc=loc_2, size=size_test)
        
        # 4.b. Запускаю критерий.
        _, _, _, left_bound, right_bound = absolute_ttest(control, test)

        # 4.c. Проверяю, лежит ли истинная разница средних в доверительном интервале.
        if left_bound > 0 or right_bound < 0:
            bad_cnt += 1

    # 5. Строю доверительный интервал для конверсии ошибок у критерия.
    left_ci, right_ci = proportion_confint(count=bad_cnt, nobs=N)
    return bad_cnt / N, left_ci, right_ci

In [18]:
alpha_real, left_ci, right_ci = experiment_1(N=1000, size_control=500, size_test=600, loc_1=1000, loc_2=1000)

print(f"Реально достигнутый уровень значимости = {round(alpha_real, 4)}",
      f"Левая и правая граница доверительного интервала для alpha_real = {round(left_ci, 4), round(right_ci, 4)}",
      sep='\n\n')

Реально достигнутый уровень значимости = 0.059

Левая и правая граница доверительного интервала для alpha_real = (0.0444, 0.0736)


Сначала валидируйте критерий на искусственных датасетах, чтобы точнее оценить, не ошибается ли он на простых распределениях. И лишь после этого переходите к датасетам на реальных данных.

Таким образом мы прогоняем наши критерии только на А/А-тестах. Но на самом деле можно эмулировать и A/B-тесты. Казалось бы, зачем, но про это расскажем позже.

Теперь поговорим про относительный Т-тест критерий.

## Относительный Т-тест.

In [19]:
# 2. Создание тестируемого критерия.
def relative_ttest(control, test, alpha=0.05):
    mean_control = np.mean(control)
    mean_test = np.mean(test)
    var_mean_control  = np.var(control) / len(control)
    var_mean_test  = np.var(test) / len(test)

    difference_mean = mean_test - mean_control
    difference_mean_var = var_mean_control + var_mean_test
    difference_distribution = sps.norm(loc=difference_mean, scale=np.sqrt(difference_mean_var))

    left_bound, right_bound = difference_distribution.ppf([alpha / 2, 1 - alpha / 2])
    left_bound = left_bound / np.mean(control)   # Деление на среднее в контроле
    right_bound = right_bound / np.mean(control) # Деление на среднее в контроле

    ci_length = (right_bound - left_bound)
    pvalue = 2 * min(difference_distribution.cdf(0), difference_distribution.sf(0))
    effect = difference_mean
    return ExperimentComparisonResults(pvalue, effect, ci_length, left_bound, right_bound)

In [20]:
def experiment_2(N=10000, size_control=1500, size_test=1500, loc_1=1, loc_2=1):
    # 3. Заводим счётчик.
    bad_cnt = 0

    # 4. Цикл проверки.
    for i in range(N):
        # 4.a. Тестирую A/A-тест.
        control = sps.expon.rvs(loc=loc_1, size=size_control)
        test = sps.expon.rvs(loc=loc_2, size=size_test)
        
        # 4.b. Запускаю критерий.
        _, _, _, left_bound, right_bound = relative_ttest(control, test)

        # 4.c. Проверяю, лежит ли истинная разница средних в доверительном интервале.
        if left_bound > 0 or right_bound < 0:
            bad_cnt += 1

    # 5. Строю доверительный интервал для конверсии ошибок у критерия.
    left_ci, right_ci = proportion_confint(count=bad_cnt, nobs=N)
    return bad_cnt / N, left_ci, right_ci

In [21]:
alpha_real, left_ci, right_ci = experiment_2(N=1000, size_control=500, size_test=600, loc_1=1000, loc_2=1000)

print(f"Реально достигнутый уровень значимости = {round(alpha_real, 4)}",
      f"Левая и правая граница доверительного интервала для alpha_real = {round(left_ci, 4), round(right_ci, 4)}",
      sep='\n\n')

Реально достигнутый уровень значимости = 0.054

Левая и правая граница доверительного интервала для alpha_real = (0.04, 0.068)


## Методы борьбы с выбросами в данных

Рассмотрим следующие выборки в контроле и тесте:

In [22]:
sample_control = [3] * 30 + [10] * 30 + [200] * 10 + [1200]
sample_test    = [8] * 30 + [20] * 30 + [100] * 10 + [1000]

sample_control = np.array(sample_control) + sps.norm().rvs(len(sample_control)) # придадим шума
sample_test    = np.array(sample_test) + sps.norm().rvs(len(sample_test)) # придадим шума

Итак, в данном случае есть 4 группы пользователей. Как мы видим, в группах с низкой метрикой значения на тесте значительно выше (в относительном смысле), чего нельзя сказать о группе с высокой метрикой - там, наоборот, значения на тесте ниже!

Давайте попробуем применить несколько критериев и проинтерпретируем их:

In [23]:
sps.ttest_ind(sample_control, sample_test, alternative='less')

Ttest_indResult(statistic=0.45076353541881664, pvalue=0.6735713334513904)

Как мы видим, гипотеза о том, что метрика в тесте выше, не подтверждается. Здесь мы использовали `ttest_ind` - это Т-тест Уэлча. Подробнее можно посмотреть вот: [здесь](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html)

А теперь примением критерий Манна-Уитни и тот же Т-тест, но уже предварительно применив к выборкам функцию `np.log(x + 1)`

In [24]:
sps.mannwhitneyu(sample_control, sample_test)

MannwhitneyuResult(statistic=1566.0, pvalue=9.927514766857255e-05)

In [25]:
sps.ttest_ind(np.log(sample_control + 1), np.log(sample_test + 1), alternative='less')

Ttest_indResult(statistic=-2.7161797510335854, pvalue=0.0037184565231908806)

А теперь у нас гипотеза подтверждается. Т.е метрика на тесте значимо выше, чем на контроле. Тритмент есть, выкатываем фичу!

Напоследок убедимся, что метрика на тесте и правда выше, чем на контроле:

In [26]:
np.mean(sample_control), np.mean(sample_test)

(50.340143144786325, 39.906954925606826)

Да уж, здесь метрика на тесте значительно ниже, чем на контроле. Но ведь два последних теста говорят нам об обратном.

Что делать?

Проблема в том, что тест Манна-Уитни проверяет совершенно другую гипотезу (об этом можно посмотреть в ноутбуке про критерий Манна-Уитни). Аналогично при логарифмировании у нас снова нулевая гипотеза - это совсем не то, что нужно.

В первом же тесте `ttest_ind(alternative='less)` мы вообще проверяли другую гипотезу. У нас метрика на контроле явно доминирует над метрикой на тесте. Давайте посмотрим, значимо ли это происходит в настоящий момент времени или нет:

In [27]:
sps.ttest_ind(sample_control, sample_test, alternative='greater')

Ttest_indResult(statistic=0.45076353541881664, pvalue=0.3264286665486095)

Нет, пока что это происходит не значимо. Возможно, если выборка станет побольше, то мы все-таки увидим значимую разницу (ведь она и правда довольно большая).

Увеличим выборки, сохранив пропорции:

In [28]:
sample_control = [3] * 30 * 100 + [10] * 30 * 100 + [200] * 10 * 100 + [1200] * 100
sample_test    = [8] * 30 * 100 + [20] * 30 * 100 + [100] * 10 * 100 + [1000] * 100

sample_control = np.array(sample_control) + sps.norm().rvs(len(sample_control)) # придадим шума
sample_test    = np.array(sample_test) + sps.norm().rvs(len(sample_test)) # придадим шума

Посмотрим на метрики (тест/контроль):

In [29]:
np.mean(sample_control), np.mean(sample_test)

(50.56518580346066, 39.9986873187836)

А теперь посмотрим, значимо ли наша метрика ухудшилась после добавления фичи, т.е на тесте:

In [30]:
sps.ttest_ind(sample_control, sample_test, alternative='greater')

Ttest_indResult(statistic=4.597886281757246, pvalue=2.1524103725766345e-06)

Да! Стоило лишь немного подождать.

Какой вывод мы можем сделать?
- Не нужно использовать критерии, которые проверяют совершенно другую гипотезу. Т.е тест Манна-Уитни и логарифмирование + Т-тест не дадут вам правильный ответ + запутают в интерпретации.
- Иногда для прокраса теста нужно время, поэтому достаточно подождать нужного размера выборки (об этом более подробно во второй части)
- Проверяйте бизнес-метрики на тесте и контроле перед самим тестом. На глаз можно понять, как вообще влияет наша фича. Затем уже можно запускать критерий.


В данном кейсе у нас были очевидные выбросы (1200 и 1000 на контроле и тесте соответственно). Давайте посмотрим, как можно бороться с выбросами и откидывать топ n%.

## Убрать топ 1% пользователей с максимальной метрикой в тесте и контроле

In [43]:
def experiment(N=10000, size_control=1500, size_test=1500, loc_1=1, loc_2=1):
    # 3. Заводим счётчик.
    bad_cnt = 0

    # 4. Цикл проверки.
    for i in range(N):
        # 4.a. Тестирую A/A-тест.
        control = sps.expon.rvs(loc=loc_1, size=size_control)
        test = sps.expon.rvs(loc=loc_2, size=size_test)

        outlier_control_filter = np.quantile(control, 0.99)
        outlier_test_filter = np.quantile(test, 0.99)
        
        control = control[control < outlier_control_filter] # убираем топ 1% в контроле
        test    = test[test < outlier_test_filter] # убираем топ 1% в тесте
        
        # 4.b. Запускаю критерий.
        _, _, _, left_bound, right_bound = relative_ttest(control, test)

        # 4.c. Проверяю, лежит ли истинная разница средних в доверительном интервале.
        if left_bound > 0 or right_bound < 0:
            bad_cnt += 1

    # 5. Строю доверительный интервал для конверсии ошибок у критерия.
    left_ci, right_ci = proportion_confint(count=bad_cnt, nobs=N)
    return bad_cnt / N, left_ci, right_ci

In [44]:
alpha_real, left_ci, right_ci = experiment(N=30000, size_control=500, size_test=600, loc_1=1000, loc_2=1000)

print(f"Реально достигнутый уровень значимости = {round(alpha_real, 4)}",
      f"Левая и правая граница доверительного интервала для alpha_real = {round(left_ci, 4), round(right_ci, 4)}",
      sep='\n\n')

Реально достигнутый уровень значимости = 0.069

Левая и правая граница доверительного интервала для alpha_real = (0.0662, 0.0719)


Как мы видим, реально достигнутый уровень значимости далек от 0.05. Почему так происходит?

**Первая проблема:**
Дело в том, что из-за выбросов у нас выборочные квантили могут иметь совершенно разный порядок. Поэтому один из кейсов, что в одной выборке значения могут быть меньше 2000, а в другой - меньше 3000.

Чтобы исправить этот недостаток, можно брать **одну** квантиль для теста и контроля, посчитанную на всём тесте или на всём контроле или на объединенной выборке теста и контроля. На А/А-тестах такая вещь работает. 

In [41]:
def experiment(N=10000, size_control=1500, size_test=1500, loc_1=1, loc_2=1):
    # 3. Заводим счётчик.
    bad_cnt = 0

    # 4. Цикл проверки.
    for i in range(N):
        # 4.a. Тестирую A/A-тест.
        control = sps.expon.rvs(loc=loc_1, size=size_control)
        test = sps.expon.rvs(loc=loc_2, size=size_test)

        outlier_filter = np.quantile(np.concatenate([control, test]), 0.99)
        
        control = control[control < outlier_filter] # убираем топ 1% в контроле
        test    = test[test < outlier_filter] # убираем топ 1% в тесте
        
        # 4.b. Запускаю критерий.
        _, _, _, left_bound, right_bound = relative_ttest(control, test)

        # 4.c. Проверяю, лежит ли истинная разница средних в доверительном интервале.
        if left_bound > 0 or right_bound < 0:
            bad_cnt += 1

    # 5. Строю доверительный интервал для конверсии ошибок у критерия.
    left_ci, right_ci = proportion_confint(count=bad_cnt, nobs=N)
    return bad_cnt / N, left_ci, right_ci

In [42]:
alpha_real, left_ci, right_ci = experiment(N=30000, size_control=500, size_test=600, loc_1=1000, loc_2=1000)

print(f"Реально достигнутый уровень значимости = {round(alpha_real, 4)}",
      f"Левая и правая граница доверительного интервала для alpha_real = {round(left_ci, 4), round(right_ci, 4)}",
      sep='\n\n')

Реально достигнутый уровень значимости = 0.0502

Левая и правая граница доверительного интервала для alpha_real = (0.0477, 0.0527)


Такая вещь работает, потому что мы выбираем один порог для двух выборок. При этом обьединяя их - мы учитываем как выбросы в контроле, так и выбросы в тесте. Тем самым находя что-то среднее (ведь выборки из одного распределения).

Часть 2

CUPED (Controlled-experiment Using Pre-Experiment Data) — очень популярный в последнее время метод уменьшения вариации.

Основная идея метода такова: давайте вычтем что-то из теста и из контроля так, чтобы математическое ожидание разницы новых величин осталось таким же, как было, а дисперсия уменьшилась.

Более формально:

Пусть $T$ - метрика на тестовой выборке, а $C$ - на контрольной. Будем как обычно проверять гипотезу $H_0 = \{\mathbb{E}(T) = \mathbb{E}(C)\}$, против сложной классической альтернативы $H_1 = \{\mathbb{E}(T) \neq \mathbb{E}(C)\}$. Итак, рассмотрим новые случайные величины $T^{'}$ и $C^{'}$ такие, что
\begin{align*}
    T^{'} : = T - \theta \cdot A \text{ и}\\
    C^{'} : = C - \theta \cdot B,
\end{align*}
где $\theta$ - некоторая константа, а $A$ и $B$ независимые случайные величины такие, что $\mathbb{E}(A) = \mathbb{E}(B)$. Тогда верно, что
\begin{align*}
    \mathbb{E}(T^{'} - C^{'}) &= \mathbb{E}(T^{'}) - \mathbb{E}(C^{'}) \\
    &= (\mathbb{E}(T) - \mathbb{E}(C)) - \theta \cdot (\mathbb{E}(A) - \mathbb{E}(B)) \\
    &= \mathbb{E}(T) - \mathbb{E}(C)
\end{align*}
Т.е матожидание разницы новых метрик такое же, как матожидание разницы старых метрик, а также $T^{'} и C^{'}$ независимы. Поэтому можно рассматривать аналогичную задачу, т.е проверять гипотезу $H_0 = \{\mathbb{E}(T^{'}) = \mathbb{E}(C^{'})\}$, против альтернативы $H_1 = \{\mathbb{E}(T^{'}) \neq \mathbb{E}(C^{'})\}$.

Теперь наша цель - понижение дисперсии разницы новых метрик, т.е 
\begin{align*}
    \argmin_{\theta} \{\mathbb{D}(T^{'} - C^{'})\} \Leftrightarrow \\
    \argmin_{\theta} \{\mathbb{D}((T - C) - \theta \cdot (A - B))\} \Leftrightarrow \\
    \argmin_{\theta} \{\mathbb{D}(T - C) + {\theta}^2 \cdot \mathbb{D}(A - B) - 2 \cdot \theta \cdot \mathrm{Cov}[T - C, A - B]\} 
\end{align*}

Легко заметить, что мы получили квадратное уравнение в зависимости от $\theta$. Очевидно, минимум достигается в точке
\begin{align*}
    \theta^{*} = \frac{\mathrm{Cov}[T - C, A - B]}{\mathbb{D}(A - B)}.
\end{align*}

Также отметим, что дисперсия разницы новых метрик при $\theta = {\theta}^{*}$ равна 
\begin{align*}
    \mathbb{D}(T^{'} - C^{'}) &= \mathbb{D}(T - C) + {\left(\frac{\mathrm{Cov}[T - C, A - B]}{\mathbb{D}(A - B)}\right)}^2 \cdot \mathbb{D}(A - B) - 2 \cdot \frac{\mathrm{Cov}[T - C, A - B]}{\mathbb{D}(A - B)} \cdot \mathrm{Cov}[T - C, A - B] \\
    &= \mathbb{D}(T - C) - {\left(\frac{\left(\mathrm{Cov}[T - C, A - B]\right)^2}{\mathbb{D}(A - B)}\right)} = \mathbb{D}(T - C) - \mathbb{D}(T - C) \cdot {\left(\frac{\left(\mathrm{Cov}[T - C, A - B]\right)^2}{\mathbb{D}(A - B) \cdot \mathbb{D}(T - C)}\right)} \\
    &= \mathbb{D}(T - C) \cdot \left(1 - {\left(\frac{\left(\mathrm{Cov}[T - C, A - B]\right)^2}{\mathbb{D}(A - B) \cdot \mathbb{D}(T - C)}\right)}\right) = \mathbb{D}(T - C) \cdot (1 - {\alpha}^2),
\end{align*}
где $\alpha = \mathrm{Corr}[T - C, A - B]$.


Отсюда следует несколько фактов:
1. Дисперсия у разницы новых метрик всегда ниже, чем дисперсия разницы старых метрик.
2. Если мы хотим еще сильнее уменьшить дисперсию разницы новых метрик, то нужно делать так, чтобы $T - C$ и $A - B$ сильно коррелировали. Этого можно добиться тем, что на предпериоде мы будем случайно бить на тест и контроль и в качестве $A$ будем брать тест на предпериоде, а в качестве B - контроль на предпериоде. Понятно также, что математические ожидания у $A$ и $B$ должны быть равны и так будет тогда и только тогда, когда A/A тест будет успешно проходить! (а он и обязан успешно проходить, иначе дальнейшие действия бессмысленны) Кроме значения метрики на предпериоде можно использовать результаты ML-модели, обученной предсказывать истинные значения метрик без влияния тритмента. С хорошей моделью можно достичь большего уменьшения дисперсии.

Теперь можно провести симмуляцию и посмотреть, как сильно мы будем выигрывать в дисперсии разницы новых метрик:

Напишем для этого функцию `cuped_ttest`:

In [67]:
# 2. Создание функции cuped_ttest
def cuped_ttest(control, test, control_before, test_before):
    theta = (np.cov(control, control_before)[0, 1] + np.cov(test, test_before)[0, 1]) \
          / (np.var(test_before) + np.var(control_before))
    
    new_test = test - theta * test_before
    new_control = control - theta * control_before
    
    return absolute_ttest(control=new_control, test=new_test)

Проведем симмуляцию A/A теста для построенных cuped-метрик:

In [68]:
# 3. Заводим счётчик.
bad_cnt = 0

# 4. Цикл проверки.
N = 30000
for i in tqdm_notebook(range(N)):
    # 4.a. Тестирую A/A-тест.
    control_before = sps.expon(scale=1000).rvs(1000)
    test_before = sps.expon(scale=1000).rvs(1000)

    control = control_before + sps.norm(loc=0, scale=100).rvs(1000)
    test = test_before + sps.norm(loc=0, scale=100).rvs(1000)
    test *= 1

    # 4.b. Запускаю критерий.
    _, _, _, left_bound, right_bound = cuped_ttest(control, test, control_before, test_before)
    
    # 4.c. Проверяю, лежит ли истинная разница средних в доверительном интервале. Если не лежит - увеличиваю счетчик на 1.
    if left_bound > 100 or right_bound < 100:
        bad_cnt += 1
        
# 5. Строю доверительный интервал для конверсии ошибок у критерия.
left_real_level, right_real_level = proportion_confint(count = bad_cnt, nobs = N, alpha=0.05, method='wilson')

# Результат.
print(f"Реальный уровень значимости: {round(bad_cnt / N, 4)}",
      f"Доверительный интервал: [{round(left_real_level, 4)}, {round(right_real_level, 4)}]", sep='\n')

  0%|          | 0/30000 [00:00<?, ?it/s]

Реальный уровень значимости: 1.0
Доверительный интервал: [0.9999, 1.0]


Давайте ещё посмотрим, на сколько в искусственном примере уменьшилась ширина доверительного интервала по сравнению с обычным T-test:

In [66]:
# 3. Заводим счётчик.
bad_cnt = 0

ci_length_cuped_ttest_arr = [] # здесь будем хранить длины доверительных интервалов для cuped_ttest
ci_length_ttest_arr = [] # здесь будем хранить длины доверительных интервалов для ttest

# 4. Цикл проверки.
N = 30000
for i in tqdm_notebook(range(N)):
    # 4.a. Тестирую A/A-тест.
    control_before = sps.expon(scale=1000).rvs(1000)
    test_before = sps.expon(scale=1000).rvs(1000)

    control = control_before + sps.norm(loc=0, scale=100).rvs(1000)
    test = test_before + sps.norm(loc=0, scale=100).rvs(1000)
    test *= 1

    # 4.b. Запускаю критерий.
    _, _, ci_length_cuped_ttest, _, _ = cuped_ttest(control, test, control_before, test_before)
    _, _, ci_length_ttest, _, _ = absolute_ttest(control, test)

    # 4.c. Добавляем в списки полученные длины

    ci_length_cuped_ttest_arr.append(ci_length_cuped_ttest)
    ci_length_ttest_arr.append(ci_length_ttest)

        
# 5. Строю доверительный интервал для конверсии ошибок у критерия.
left_real_level, right_real_level = proportion_confint(count = bad_cnt, nobs = N, alpha=0.05, method='wilson')

coeff = round(np.array(ci_length_ttest_arr).mean() / np.array(ci_length_cuped_ttest_arr).mean(), 2)
# Результат.
print(f"Доверительный интервал для cuped_ttest относительно доверительного интервала для ttest уменьшился в {coeff} раз.")

  0%|          | 0/30000 [00:00<?, ?it/s]

Доверительный интервал для cuped_ttest относительно доверительного интервала для ttest уменьшился в 10.05 раз.
